<left>FINM 32000 - Numerical Methods</left>
<left>Spring 2023</left>
<br>
<h1><center> Homework 3 </center></h1>
<center>Due - 23:59 [CST] April 21st, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

#### Imports

In [1]:
import numpy as np

#### Helper Functions

In [2]:
class Vasicek:
    
    def __init__(self,kappa,theta,sigma): 
        self.kappa=kappa
        self.theta=theta
        self.sigma=sigma

In [3]:
class Bond:
    
    def __init__(self, T):
        self.T=T


In [12]:
class FDexplicit:
    
    def __init__(self, rMax, rMin, deltar, deltat, useUpwind):         
        self.rMax=rMax
        self.rMin=rMin
        self.deltar=deltar
        self.deltat=deltat
        self.useUpwind=useUpwind
        
    
    def price_bond_vasicek(self,contract,dynamics):
    # You complete the coding of this function
    #
    # Returns array of all initial short rates,
    # and the corresponding array of zero-coupon
    # T-maturity bond prices

        T = contract.T
        N = round(T/self.deltat)
        if abs(N-T/self.deltat) > 1e-12:
            raise ValueError("Bad delta t")

        r = np.arange(self.rMax,self.rMin-self.deltar/2,-self.deltar)
        #I'm making the FIRST indices of the array correspond to HIGH levels of r
        bondprice = np.ones(np.size(r))

        nu = dynamics.kappa * (dynamics.theta - r)
        sigma = dynamics.sigma

        if self.useUpwind:
            qu =  (sigma**2 * self.deltat/(self.deltar))/2 + np.maximum(nu, 0) * self.deltat/self.deltar
            qd =  (sigma**2 * self.deltat/(self.deltar))/2 - np.minimum(nu, 0) * self.deltat/self.deltar
            qm =  1 - qu - qd
        else:
            qu =  (sigma**2 * self.deltat/(self.deltar) + nu * self.deltat/self.deltar)/2
            qd =  (sigma**2 * self.deltat/(self.deltar) - nu * self.deltat/self.deltar)/2
            qm =  1 - (sigma**2 * self.deltat/(self.deltar))

        for t in np.arange(N-1,-1,-1)*self.deltat:
            bondprice=1/(1+r*self.deltat)*(qd*np.roll(bondprice,-1)+qm*bondprice+qu*np.roll(bondprice,1))

            # It is not obvious in this case,
            # what boundary conditions to use at the top and bottom
            # so let us assume "linearity" boundary conditions
            bondprice[0]=2*bondprice[1]-bondprice[2]
            bondprice[-1]=2*bondprice[-2]-bondprice[-3]

        return (r, bondprice)

# Problem 1.

In [6]:
hw3dynamics = Vasicek(kappa=3, theta=0.05, sigma=0.03)

In [7]:
hw3contract = Bond(T=5)

## (a)

Using Ito's third rule:

$$
dC(r_t, t) = C'(r_t, t) dr_t +
\dot{C}(r_t, t) dt
+ \frac{1}{2} C''(r_t, t) d<r>_t
$$

We were also given from the question that

$$
dr_t = \alpha(r_t, t) dt + \beta(r_t, t) dW_t
$$

Combining the two:

$$
\begin{aligned}
dC(r_t, t) &= C'(r_t, t) \left(
\alpha(r_t, t) dt + \beta(r_t, t) dW_t
\right) +\dot{C}(r_t, t) dt
+ \frac{1}{2} (\beta(r_t, t))^2 \cdot C''(r_t, t) dt \\
&= \left(
\dot{C}(r_t, t) +
\alpha(r_t, t) \cdot C'(r_t, t) +
\frac{1}{2} (\beta(r_t, t))^2 \cdot C''(r_t, t)
\right) dt +
\beta(r_t, t) \cdot C'(r_t, t) dW_t
\end{aligned}
$$

Since the drift was set to be equal to $rC$, the PDE for $C(r_t, t)$ can be set as below:

$$
r_t C(r_t, t) =
\dot{C}(r_t, t) +
\alpha(r_t, t) \cdot C'(r_t, t) +
\frac{1}{2} (\beta(r_t, t))^2 \cdot C''(r_t, t)
$$

## (b)

With the risk-neutral Vasicek dynamics of $r_t$:

$$
dr_t = \kappa (\theta - r_t) dt + \sigma dW_t
$$

the PDE of $C(r_t, t)$ becomes:

$$
r_t C(r_t, t) =
\dot{C}(r_t, t) +
\kappa (\theta - r_t) \cdot C'(r_t, t) +
\frac{1}{2} \sigma^2 \cdot C''(r_t, t)
$$

Moreover, the finite difference solution for $C^{j}_{n}$ may be derived from:

$$
\begin{aligned}
q_u &= \frac{1}{2} \left[
\frac{\sigma^2 \Delta t}{(\Delta r)^2} +
\kappa (\theta - r_t) \frac{\Delta t}{\Delta r}
\right] \\
q_m &= 1 - \frac{\sigma^2 \Delta t}{(\Delta r)^2} \\
q_d &= \frac{1}{2} \left[
\frac{\sigma^2 \Delta t}{(\Delta r)^2} -
\kappa (\theta - r_t) \frac{\Delta t}{\Delta r}
\right]
\end{aligned}
$$

as

$$
C^j_n = \frac{1}{1 + r_t \Delta t}
(q_u C^{j + 1}_{n + 1} +
q_m C^{j}_{n + 1} +
q_d C^{j - 1}_{n + 1})
$$

In [8]:
hw3FD = FDexplicit(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=False)

In [9]:
(r, bondprice) = hw3FD.price_bond_vasicek(hw3contract,hw3dynamics)

In [10]:
np.set_printoptions(precision=4,suppress=True)
displayrows=(r < 0.15 + hw3FD.deltar/2) & (r > 0.0 - hw3FD.deltar/2)

In [11]:
print(np.stack((r, bondprice),1)[displayrows])

[[ 1.5000e-01  1.5847e+24]
 [ 1.4000e-01 -6.2015e+23]
 [ 1.3000e-01 -6.4913e+22]
 [ 1.2000e-01  1.5446e+22]
 [ 1.1000e-01  1.4539e+21]
 [ 1.0000e-01 -1.9860e+20]
 [ 9.0000e-02 -1.4779e+19]
 [ 8.0000e-02  9.8659e+17]
 [ 7.0000e-02  4.3241e+16]
 [ 6.0000e-02 -7.7557e+14]
 [ 5.0000e-02  6.6014e+12]
 [ 4.0000e-02 -1.1135e+16]
 [ 3.0000e-02  4.1447e+17]
 [ 2.0000e-02  1.3710e+19]
 [ 1.0000e-02 -1.3958e+20]
 [-3.3307e-16 -2.6921e+21]]


## (c)

When $\kappa (\theta - r_j) \geq 0$:

$$
\frac{C^{j}_{n + 1} - C^{j}_{n}}{\Delta t} +
\kappa (\theta - r_j) \frac{C^{j + 1}_{n + 1} - C^{j}_{n + 1}}{\Delta x} +
\frac{1}{2} \sigma^2 \frac{C^{j + 1}_{n + 1} - 2C^{j}_{n + 1} + C^{j - 1}_{n + 1}}{(\Delta x)^2}
= r C^{j}_{n}
$$

From this we can get:

$$
\begin{aligned}
q_u &= \frac{1}{2} \left[
\frac{\sigma^2 \Delta t}{(\Delta r)^2} \right] +
\kappa (\theta - r_t) \frac{\Delta t}{\Delta r} \\
q_m &= 1 - \frac{\sigma^2 \Delta t}{(\Delta r)^2} -
\kappa (\theta - r_t) \frac{\Delta t}{\Delta r} \\
q_d &= \frac{1}{2} \left[
\frac{\sigma^2 \Delta t}{(\Delta r)^2}
\right]
\end{aligned}
$$

When $\kappa (\theta - r_j) < 0$:

$$
\frac{C^{j}_{n + 1} - C^{j}_{n}}{\Delta t} +
\kappa (\theta - r_j) \frac{C^{j}_{n + 1} - C^{j - 1}_{n + 1}}{\Delta x} +
\frac{1}{2} \sigma^2 \frac{C^{j + 1}_{n + 1} - 2C^{j}_{n + 1} + C^{j - 1}_{n + 1}}{(\Delta x)^2}
= r C^{j}_{n}
$$

From this we can get:

$$
\begin{aligned}
q_u &= \frac{1}{2} \left[
\frac{\sigma^2 \Delta t}{(\Delta r)^2} \right] \\
q_m &= 1 - \frac{\sigma^2 \Delta t}{(\Delta r)^2} +
\kappa (\theta - r_t) \frac{\Delta t}{\Delta r} \\
q_d &= \frac{1}{2} \left[
\frac{\sigma^2 \Delta t}{(\Delta r)^2} \right] -
\kappa (\theta - r_t) \frac{\Delta t}{\Delta r}
\end{aligned}
$$

In [13]:
hw3FD_upwind = FDexplicit(rMax=0.35,rMin=-0.25,deltar=0.01,deltat=0.01,useUpwind=True)

In [14]:
(r_upwind, bondprice_upwind) = hw3FD_upwind.price_bond_vasicek(hw3contract, hw3dynamics)

In [15]:
np.set_printoptions(precision=4, suppress=True)
displayrows = (r < 0.15 + hw3FD_upwind.deltar / 2) & (r > 0.0 - hw3FD_upwind.deltar / 2)

In [16]:
print(np.stack((r_upwind, bondprice_upwind), 1)[displayrows])

[[ 0.15    0.7534]
 [ 0.14    0.7559]
 [ 0.13    0.7584]
 [ 0.12    0.7609]
 [ 0.11    0.7635]
 [ 0.1     0.766 ]
 [ 0.09    0.7686]
 [ 0.08    0.7711]
 [ 0.07    0.7737]
 [ 0.06    0.7763]
 [ 0.05    0.7789]
 [ 0.04    0.7815]
 [ 0.03    0.7841]
 [ 0.02    0.7867]
 [ 0.01    0.7893]
 [-0.      0.792 ]]


## (d)

By Taylor's theorem, as $h \rightarrow 0$:

$$
f(x+h) = f(x) + f'(x)(x+h-x) + \frac{f''(x)}{2!}(x+h-x)^2 + O(h^3) = f(x) + f'(x)(h) + \frac{f''(x)}{2!}(h)^2 + O(h^3)\\
f(x-h) = f(x) + f'(x)(x-h-x) + \frac{f''(x)}{2!}(x-h-x)^2 + O(h^3) = f(x) + f'(x)(-h) + \frac{f''(x)}{2!}(h)^2 + O(h^3)
$$

Therefore,

